In [1]:
import numpy as np
import scipy.io as sio
import numpy as np
import h5py
    
def LagGen(x,b,a):
    l, m = x.shape
    x0 = np.pad(x, ((b, a), (0,0)), 'edge')
    o = np.zeros((l,b+a+1,m))
    for i in range(l):
        for j in range(m):
            o[i,:,j] = x0[i:i+a+b+1,j]
    return o

In [3]:
##HG+LFP
b=0
a=30

#importing neural data
mat = sio.loadmat('LIJ123_pitch_Ce_Test.mat')
neural_data_HG = np.transpose(mat['neural_data_HG'])
neural_data_LFP = np.transpose(mat['neural_data_LFP'])

#importing vocoder parameters
f0 = np.log10(np.transpose(mat['f0'])+1)
vuv = np.transpose(mat['vuv'])
aperiodicity = np.log10(-np.transpose(mat['aperiodicity'])+1)
spectrogram = np.transpose(mat['spectrogram'])
spectrogram[spectrogram>1] = 1
spectrogram = np.power(spectrogram,.1) #compressing
#spectrogram = np.log10(spectrogram+1) #study this later

#performing lag+stacking neural data
print('Generating 3D lags for HighGamma...')
shft_neural_3d_HG = LagGen(neural_data_HG,b,a)
shft_neural_3d_HG = shft_neural_3d_HG.astype('float32')
print('Generating 3D lags for LFP...')
shft_neural_3d_LFP = LagGen(neural_data_LFP,b,a)
shft_neural_3d_LFP = shft_neural_3d_LFP.astype('float32')
shp = shft_neural_3d_HG.shape
shft_neural_4d = np.zeros((shp[0],shp[1],shp[2],2),dtype='float32')
shft_neural_4d[:,:,:,0] = shft_neural_3d_HG
shft_neural_4d[:,:,:,1] = shft_neural_3d_LFP

print('Input shape_4d:', shft_neural_4d.shape)
print('Output shape (f0):', f0.shape)
print('Output shape (vuv):', vuv.shape)
print('Output shape (aperiodicity):', aperiodicity.shape)
print('Output shape (spectrogram):', spectrogram.shape)

print('Saving data...')
#sio.savemat('Data_Decoding_128fBins_LIJ120_all_elecs_Train.mat', mdict={'features': features, 'shft_neural_4d': shft_neural_4d})
h5f = h5py.File('Data_Decoding_LIJ123_HGaLFP_pitch_Test.h5','w')
h5f.create_dataset('f0', data=f0)
h5f.create_dataset('vuv', data=vuv)
h5f.create_dataset('aperiodicity', data=aperiodicity)
h5f.create_dataset('spectrogram', data=spectrogram)
h5f.create_dataset('shft_neural_4d', data=shft_neural_4d)
h5f.close()
print('Saving done.')

Generating 3D lags for HighGamma...
Generating 3D lags for LFP...
Input shape_4d: (4008, 31, 61, 2)
Output shape (f0): (4008, 1)
Output shape (vuv): (4008, 1)
Output shape (aperiodicity): (4008, 1)
Output shape (spectrogram): (4008, 513)
Saving data...
Saving done.
